In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Bidirectional, Dense, Conv1D, LSTM, Flatten, Attention
from utilz import *
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

2022-10-12 14:34:08.946056: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-12 14:34:09.030679: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-12 14:34:09.052155: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-12 14:34:09.429371: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2022-10-12 14:34:10.109115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 14:34:10.137802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-12 14:34:10.137913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# load data

In [2]:
acoustic = load_features('./data/acoustic_wav2vec.pkl')
# acoustic = load_features('./data/acoustic_opensml.pkl')
label = load_features('./data/labels.pkl')

# Model 1

In [3]:
x = Input((128,512))
h = Bidirectional(LSTM(32, dropout=0.3))(x)
res = Dense(3, 'softmax')(h)
    
model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

2022-10-03 19:54:33.121553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 19:54:33.146913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 19:54:33.147031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-03 19:54:33.147836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 512)]        0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               139520    
 l)                                                              
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 139,715
Trainable params: 139,715
Non-trainable params: 0
_________________________________________________________________


In [4]:
callback_list = [ModelCheckpoint(filepath='./res/A_model_BLSTM_wav2vec.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(acoustic['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(acoustic['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
 1/86 [..............................] - ETA: 2:53 - loss: 1.4992 - acc: 0.1875

2022-10-03 19:54:48.317819: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2022-10-03 19:54:48.395922: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


86/86 [==============================] - ETA: 0s - loss: 1.0086 - acc: 0.5007

INFO:tensorflow:Assets written to: ./res/A_model_BLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./res/A_model_BLSTM_wav2vec.tf/assets


86/86 [==============================] - 9s 82ms/step - loss: 1.0086 - acc: 0.5007 - val_loss: 0.9889 - val_acc: 0.5395
Epoch 2/30
86/86 [==============================] - 1s 17ms/step - loss: 0.9166 - acc: 0.5643 - val_loss: 0.9917 - val_acc: 0.5307
Epoch 3/30
86/86 [==============================] - 1s 17ms/step - loss: 0.8547 - acc: 0.6053 - val_loss: 1.0426 - val_acc: 0.4715
Epoch 4/30
86/86 [==============================] - 1s 17ms/step - loss: 0.8145 - acc: 0.6374 - val_loss: 0.9969 - val_acc: 0.5417
Epoch 5/30
86/86 [==============================] - 1s 17ms/step - loss: 0.7498 - acc: 0.6827 - val_loss: 1.0337 - val_acc: 0.5307
Epoch 6/30
86/86 [==============================] - 1s 17ms/step - loss: 0.6914 - acc: 0.7273 - val_loss: 1.0470 - val_acc: 0.5461
Epoch 7/30
86/86 [==============================] - 1s 17ms/step - loss: 0.6231 - acc: 0.7617 - val_loss: 1.1012 - val_acc: 0.5417
Epoch 8/30
86/86 [==============================] - 1s 17ms/step - loss: 0.5800 - acc: 0.7734 

In [5]:
model = tf.keras.models.load_model('./res/A_model_BLSTM_wav2vec.tf')
pred = model.predict(np.asarray(acoustic['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

         Neg     0.5457    0.9395    0.6904       248
         Pos     0.3846    0.0714    0.1205       140
         Neu     0.2500    0.0145    0.0274        69

    accuracy                         0.5339       457
   macro avg     0.3934    0.3418    0.2794       457
weighted avg     0.4517    0.5339    0.4157       457

[[233  12   3]
 [130  10   0]
 [ 64   4   1]]


# Model 2

In [16]:
x = Input((128,512))
h = Conv1D(64, 3, 2)(x)
h = Bidirectional(LSTM(32, dropout=0.3))(h)
res = Dense(3, 'softmax')(h)
    
model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 512)]        0         
                                                                 
 conv1d_3 (Conv1D)           (None, 63, 64)            98368     
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 3)                 195       
                                                                 
Total params: 123,395
Trainable params: 123,395
Non-trainable params: 0
_________________________________________________________________


In [17]:
callback_list = [ModelCheckpoint(filepath='./res/A_model_CNN-BLSTM_wav2vec.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(acoustic['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(acoustic['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
83/86 [===========================>..] - ETA: 0s - loss: 0.9956 - acc: 0.5023

INFO:tensorflow:Assets written to: ./res/A_model_CNN-BLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./res/A_model_CNN-BLSTM_wav2vec.tf/assets


86/86 [==============================] - 8s 78ms/step - loss: 0.9907 - acc: 0.5073 - val_loss: 0.9651 - val_acc: 0.5526
Epoch 2/30
86/86 [==============================] - 1s 13ms/step - loss: 0.9459 - acc: 0.5497 - val_loss: 1.0127 - val_acc: 0.5022
Epoch 3/30
86/86 [==============================] - 1s 12ms/step - loss: 0.9208 - acc: 0.5497 - val_loss: 0.9768 - val_acc: 0.5088
Epoch 4/30
86/86 [==============================] - 1s 13ms/step - loss: 0.9052 - acc: 0.5629 - val_loss: 0.9661 - val_acc: 0.5329
Epoch 5/30
86/86 [==============================] - 1s 13ms/step - loss: 0.8743 - acc: 0.5768 - val_loss: 0.9835 - val_acc: 0.5329
Epoch 6/30
86/86 [==============================] - 1s 13ms/step - loss: 0.8258 - acc: 0.6294 - val_loss: 0.9864 - val_acc: 0.5395
Epoch 7/30
86/86 [==============================] - 1s 13ms/step - loss: 0.7984 - acc: 0.6382 - val_loss: 0.9950 - val_acc: 0.5461
Epoch 8/30
86/86 [==============================] - 1s 12ms/step - loss: 0.7524 - acc: 0.6762 

In [18]:
model = tf.keras.models.load_model('./res/A_model_CNN-BLSTM_wav2vec.tf')

pred = model.predict(np.asarray(acoustic['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

         Neg     0.5459    0.9839    0.7022       248
         Pos     0.6667    0.0143    0.0280       140
         Neu     0.1429    0.0145    0.0263        69

    accuracy                         0.5405       457
   macro avg     0.4518    0.3375    0.2521       457
weighted avg     0.5220    0.5405    0.3936       457

[[244   1   3]
 [135   2   3]
 [ 68   0   1]]


# Model 3

In [9]:
class Attention_Self(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(Attention_Self, self).__init__(**kwargs)
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.S = tf.keras.layers.Dense(1)
        self.units = units

    def call(self, features):
        # (64)
        # attention matrix (64,64) 
        # attention matrix[0] 64 featrues2 time step scores
        features_ = tf.expand_dims(features, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(features_))
        attention_weights = tf.nn.softmax(self.S(score), axis=1)
        ATTN = attention_weights * (features)
        ATTN = tf.reduce_sum(ATTN, axis=1)
        
        return ATTN
        
    def get_config(self):
        config = super(Attention_Self, self).get_config()
        config.update({"units": self.units})
        return config

In [10]:
x = Input((128,512))
h = Conv1D(64, 3, 2)(x)
h = Attention_Self(32)(h)
h = Bidirectional(LSTM(32))(h)
res = Dense(3, 'softmax')(h)
    
model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 512)]        0         
                                                                 
 conv1d_1 (Conv1D)           (None, 63, 64)            98368     
                                                                 
 attention__self (Attention_  (None, 63, 64)           4193      
 Self)                                                           
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                                 
Total params: 127,588
Trainable params: 127,588
Non-trainab

In [11]:
callback_list = [ModelCheckpoint(filepath='./res/A_model_Att-BLSTM_wav2vec.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(acoustic['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(acoustic['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
86/86 [==============================] - ETA: 0s - loss: 1.0106 - acc: 0.5329

INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec.tf/assets


86/86 [==============================] - 9s 85ms/step - loss: 1.0106 - acc: 0.5329 - val_loss: 0.9804 - val_acc: 0.5439
Epoch 2/30
83/86 [===========================>..] - ETA: 0s - loss: 0.9794 - acc: 0.5437

INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec.tf/assets


...

INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec.tf/assets


86/86 [==============================] - 7s 77ms/step - loss: 0.9725 - acc: 0.5482 - val_loss: 0.9753 - val_acc: 0.5482
Epoch 4/30
83/86 [===========================>..] - ETA: 0s - loss: 0.9645 - acc: 0.5520

INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec.tf/assets


INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec.tf/assets


86/86 [==============================] - 7s 79ms/step - loss: 0.9643 - acc: 0.5512 - val_loss: 0.9700 - val_acc: 0.5395
Epoch 5/30
86/86 [==============================] - 1s 17ms/step - loss: 0.9557 - acc: 0.5687 - val_loss: 0.9728 - val_acc: 0.5461
Epoch 6/30
86/86 [==============================] - 1s 17ms/step - loss: 0.9512 - acc: 0.5658 - val_loss: 0.9826 - val_acc: 0.5263
Epoch 7/30
86/86 [==============================] - 1s 17ms/step - loss: 0.9250 - acc: 0.6009 - val_loss: 0.9878 - val_acc: 0.5285
Epoch 8/30
86/86 [==============================] - 1s 16ms/step - loss: 0.9198 - acc: 0.5972 - val_loss: 1.0016 - val_acc: 0.5395
Epoch 9/30
86/86 [==============================] - 1s 16ms/step - loss: 0.9200 - acc: 0.5987 - val_loss: 0.9786 - val_acc: 0.5417
Epoch 10/30
86/86 [==============================] - 1s 15ms/step - loss: 0.9094 - acc: 0.5980 - val_loss: 0.9961 - val_acc: 0.5417
Epoch 11/30
86/86 [==============================] - 1s 17ms/step - loss: 0.9024 - acc: 0.605

In [12]:
model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec.tf')

pred = model.predict(np.asarray(acoustic['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 7ms/step
              precision    recall  f1-score   support

         Neg     0.5518    0.9234    0.6908       248
         Pos     0.4286    0.1286    0.1978       140
         Neu     0.0000    0.0000    0.0000        69

    accuracy                         0.5405       457
   macro avg     0.3268    0.3507    0.2962       457
weighted avg     0.4307    0.5405    0.4355       457

[[229  19   0]
 [122  18   0]
 [ 64   5   0]]


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [13]:
x = Input((128,512))
h = Conv1D(64, 3, 2)(x)
qv_attention = Attention()([h, h])

# q = GlobalAveragePooling1D()(q)
# qv_attention = GlobalAveragePooling1D()(qv_attention)
# h = Concatenate()([q, qv_attention])
h = Bidirectional(LSTM(32))(qv_attention)
res = Dense(3, 'softmax')(h)
    
model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 128, 512)]   0           []                               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 63, 64)       98368       ['input_4[0][0]']                
                                                                                                  
 attention (Attention)          (None, 63, 64)       0           ['conv1d_2[0][0]',               
                                                                  'conv1d_2[0][0]']               
                                                                                                  
 bidirectional_3 (Bidirectional  (None, 64)          24832       ['attention[0][0]']        

In [14]:
callback_list = [ModelCheckpoint(filepath='./res/A_model_Att-BLSTM_wav2vec_v2.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(acoustic['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(acoustic['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
85/86 [============================>.] - ETA: 0s - loss: 0.9860 - acc: 0.5294

INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec_v2.tf/assets


INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec_v2.tf/assets


86/86 [==============================] - 8s 78ms/step - loss: 0.9855 - acc: 0.5300 - val_loss: 0.9732 - val_acc: 0.5351
Epoch 2/30
86/86 [==============================] - 1s 14ms/step - loss: 0.9384 - acc: 0.5541 - val_loss: 0.9882 - val_acc: 0.5439
Epoch 3/30
86/86 [==============================] - ETA: 0s - loss: 0.9292 - acc: 0.5629

INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec_v2.tf/assets


INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec_v2.tf/assets


86/86 [==============================] - 6s 72ms/step - loss: 0.9292 - acc: 0.5629 - val_loss: 0.9599 - val_acc: 0.5504
Epoch 4/30
86/86 [==============================] - 1s 15ms/step - loss: 0.9421 - acc: 0.5461 - val_loss: 0.9746 - val_acc: 0.5219
Epoch 5/30
86/86 [==============================] - 1s 14ms/step - loss: 0.9386 - acc: 0.5475 - val_loss: 0.9748 - val_acc: 0.5329
Epoch 6/30
86/86 [==============================] - 1s 16ms/step - loss: 0.9245 - acc: 0.5461 - val_loss: 0.9649 - val_acc: 0.5570
Epoch 7/30
86/86 [==============================] - 1s 16ms/step - loss: 0.9074 - acc: 0.5504 - val_loss: 0.9903 - val_acc: 0.5395
Epoch 8/30
86/86 [==============================] - 1s 15ms/step - loss: 0.8952 - acc: 0.5607 - val_loss: 0.9887 - val_acc: 0.5197
Epoch 9/30
78/86 [==========================>...] - ETA: 0s - loss: 0.9265 - acc: 0.5449

INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec_v2.tf/assets


INFO:tensorflow:Assets written to: ./res/A_model_Att-BLSTM_wav2vec_v2.tf/assets


86/86 [==============================] - 6s 70ms/step - loss: 0.9307 - acc: 0.5461 - val_loss: 0.9504 - val_acc: 0.5636
Epoch 10/30
86/86 [==============================] - 1s 12ms/step - loss: 0.8956 - acc: 0.5614 - val_loss: 0.9542 - val_acc: 0.5439
Epoch 11/30
86/86 [==============================] - 1s 8ms/step - loss: 0.9030 - acc: 0.5621 - val_loss: 0.9680 - val_acc: 0.5395
Epoch 12/30
86/86 [==============================] - 1s 10ms/step - loss: 0.8919 - acc: 0.5768 - val_loss: 0.9752 - val_acc: 0.5439
Epoch 13/30
86/86 [==============================] - 1s 13ms/step - loss: 0.8754 - acc: 0.5797 - val_loss: 0.9842 - val_acc: 0.5351
Epoch 14/30
86/86 [==============================] - 1s 14ms/step - loss: 0.8908 - acc: 0.5826 - val_loss: 0.9841 - val_acc: 0.5570
Epoch 15/30
86/86 [==============================] - 1s 13ms/step - loss: 0.8797 - acc: 0.5673 - val_loss: 0.9795 - val_acc: 0.5482
Epoch 16/30
86/86 [==============================] - 1s 13ms/step - loss: 0.8889 - acc: 0

In [15]:
model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf')

pred = model.predict(np.asarray(acoustic['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

         Neg     0.5572    0.9234    0.6950       248
         Pos     0.3500    0.0500    0.0875       140
         Neu     0.2308    0.0870    0.1263        69

    accuracy                         0.5295       457
   macro avg     0.3793    0.3534    0.3029       457
weighted avg     0.4444    0.5295    0.4230       457

[[229   7  12]
 [125   7   8]
 [ 57   6   6]]
